In [1]:
import flatten_lattice as fl
import torch
from bert_models import LinearPOSBertV1
from encoding_utils import *
from transformers import AutoTokenizer
import pickle
from mask_utils import *
import json
import os
import numpy as np

device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
xlm_tok = fl.bert_tok


2022-09-24 08:32:16.283049: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-24 08:32:16.283071: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
VNUM = 11
MOD_NAME = 'bertonewayv1.pth'

# specifies files for pre-loading
LOADED = {
    'amasks': 'attmasksallv'+str(VNUM)+'.pt',
    'tmaps': 'tmapsmaskedv'+str(VNUM)+'/'
}

In [4]:
STOPS = -1

# Get examples (just use the normal lattice examples ig?)
processedgraphs = fl.get_processed_graph_data(fl.frenbase, -1, STOPS)

# get exploded candidates to generate gold labels
resarrs = [fl.get_cover_paths(p)[0] for p in processedgraphs]

# extra step for greedy 
if STOPS==1:
    processedgraphs = filter_greedy(processedgraphs)

# ensure no empty examples
clean_empty(resarrs, processedgraphs)

In [6]:
len(processedgraphs[0])

315

In [4]:
# Attention mask code (TODO needs some updating)
if os.path.exists('./torchsaved/'+LOADED['amasks']):
        print("using loaded masks")
        attmasks = torch.load('./torchsaved/'+LOADED['amasks']).to(device)
else:
    print("creating new masks")
    masktmp = [connect_mat(p) for p in processedgraphs]
    attmasks = torch.stack(masktmp).to(device)
    torch.save(attmasks, './torchsaved/'+LOADED['amasks'])

using loaded masks


In [5]:
# get tokenized inputs with posids (TODO needs an update for src/tgt format)
sents, posids = create_inputs(processedgraphs)

In [12]:
cnt = 0

for t in processedgraphs[0]:
    if t['token_idx']==0:
        print(t)
cnt

{'token_idx': 0, 'pos': 0, 'id': '0 0', 'nexts': ['581 1'], 'score': 0}
{'token_idx': 0, 'pos': 15, 'id': '0 15', 'nexts': ['41626 16'], 'score': 0}
{'token_idx': 0, 'pos': 20, 'id': '0 20', 'nexts': ['1210 21'], 'score': 0}
{'token_idx': 0, 'pos': 20, 'id': '0 20', 'nexts': ['24159 21'], 'score': 0}
{'token_idx': 0, 'pos': 15, 'id': '0 15', 'nexts': ['98 16'], 'score': 0}
{'token_idx': 0, 'pos': 17, 'id': '0 17', 'nexts': ['106 18'], 'score': 0}
{'token_idx': 0, 'pos': 19, 'id': '0 19', 'nexts': ['1210 20'], 'score': 0}
{'token_idx': 0, 'pos': 21, 'id': '0 21', 'nexts': ['33938 22'], 'score': 0}
{'token_idx': 0, 'pos': 21, 'id': '0 21', 'nexts': ['142 22'], 'score': 0}
{'token_idx': 0, 'pos': 19, 'id': '0 19', 'nexts': ['24159 20'], 'score': 0}
{'token_idx': 0, 'pos': 20, 'id': '0 20', 'nexts': ['191618 21'], 'score': 0}
{'token_idx': 0, 'pos': 20, 'id': '0 20', 'nexts': ['23 21'], 'score': 0}
{'token_idx': 0, 'pos': 21, 'id': '0 21', 'nexts': ['10 22'], 'score': 0}
{'token_idx': 0, '

0

In [13]:
bert_tok.decode(0)

'<s>'

In [10]:
processedgraphs[0][0]

{'token_idx': 0, 'pos': 0, 'id': '0 0', 'nexts': ['581 1'], 'score': 0}

In [68]:
testgraph = None
# debug the flatten_lattice method
def get_proc_graph_data(lanbase, stop=-1, msplits=-1):
    # variable tracks during lattice decoding and stops after a certain point
    global max_splits
    global splits_hit
    global testgraph
    max_splits = msplits
    base = fl.GBASE+lanbase
    paths = os.listdir(base)
    result = []
    if stop==-1:
        stop = len(paths)
    # get flattened version of lattice for each lattice in directory
    for i in range(0, stop):
        splits_hit=0
        curgraph = fl.load_graph(base+paths[i])
        testgraph = curgraph
        # get rid of duplicate nodes TODO [happens surprisingly often?]
        result.append(fl.remove_duplicates(flatten_lat(curgraph)))
        break
    return result

# flatten out lattice 
def flatten_lat(graph):
    tokdicts = []
    visited = []
    prev_contig = []
    greedy_flatten(tokdicts, visited, graph['root'], 0, prev_contig, set())
    #greedy_flat_old(tokdicts, visited, graph['root'], 0)
    return tokdicts

max_splits = -1
splits_hit = 0
# flattens graph by position, ignores </s> and en_XX tokens for greater BERT compatibility
# TODO set up to use mbart tokenization
def greedy_flatten(tdicts, visited, node, pos, prev_cont, added_ids, branch_start=None):
    global splits_hit
    if node.uid in visited:
        return
    if node.token_idx==2 or node.token_idx==250004:
        npos = pos
    else:
        node.pos = pos
        
        visited.append(node.uid)
        npos = pos+1
        s = node.token_str
        prev_cont.append(node)
    
    olen = len(tdicts)
    # we're hitting a branch or an ending, update to bert tokenization and add to visited
    # should be ok to do this since branching / merging only happens at word boundaries (presumably)
    branched = (len(node.next_scores)>1)
    end = (len(node.next_scores)==0)
    merge = end==False and node.nextlist[0].uid in visited
    if branched or merge or end:
        #print(branched, " ", merge, " ", end)
        if len(prev_cont)>0:
            errorflag = False
            
            prev_update = []
            for p in prev_cont:
                if p.uid in added_ids:
                    continue
                else:
                    prev_update.append(p)
                    added_ids.add(p.uid)
            
            if len(prev_update)>0:
                toktmp = fl.get_toklist(prev_update)
                for i in range(1, len(prev_update)):
                    if prev_update[-(i+1)].pos>=prev_update[-i].pos:
                        errorflag = True
                        break
                decstr = mbart_tok.decode(toktmp)
                if errorflag:
                    #print(decstr)
                    #print([p.pos for p in prev_update])
                    ""
                bert_toks = bert_tok(decstr).input_ids
                curpos = prev_update[0].pos
                # TODO add logic that tracks scores / next nodes
                otdlen = len(tdicts)
                for bind in range(0, len(bert_toks)):
                    b = bert_toks[bind]
                    if b==0 or b==2:
                        continue
                    nid = str(b)+" "+str(curpos)
                    # if we're at the start, add this node to next of branch node
                    if len(tdicts)==otdlen and branch_start is not None:
                        branch_start['nexts'].append(nid)

                    if bind<len(bert_toks)-1:
                        tdicts.append({
                            'token_idx':b,
                            'pos':curpos, 
                            'id': nid,
                            'nexts': [str(bert_toks[bind+1])+" "+str(curpos+1)], 
                            'score': 0
                        })
                    else:
                        tdicts.append({
                            'token_idx':b,
                            'pos':curpos, 
                            'id': str(b)+" "+str(pos),
                            'nexts': [], 
                            'score': 0
                        })
                    curpos+=1
                if merge or end:
                    splits_hit+=1
            
    if len(tdicts)>olen:
        del prev_cont
        prev_cont = []
        
    # end things early if we want to limit paths
    if max_splits>=0 and splits_hit>=max_splits:
        return 
    
    scosort = list(np.argsort(node.next_scores))
    if branched and len(tdicts)>0:
        branch_start=tdicts[-1]
    # TODO check which direction we need to go from argsort
    for i in range(0, len(scosort)):
        greedy_flatten(tdicts, visited, node.nextlist[scosort[i]], npos, prev_cont, added_ids, branch_start)


In [66]:
s = ""
cur = testgraph['root']
while len(cur.nextlist)>0:
    if len(cur.nextlist)>1:
        print([c.token_str for c in cur.nextlist])
    s+=" "+cur.token_str
    cur = cur.nextlist[0]

['on', 'Friday']
[',', '1']
['November', '1']
[',', 'st']
['2013,', '2013', 'in', 'seeking']
['in', 'seeking', 'as', 'to']
['search', 'an', 'a', 'the', 'his']
['US', 'United', 'help', 'assistance', 'U']
['assistance', 'aid', 'support', 'help']
['to', 'in']
['fight', 'combat']


In [67]:
bert_tok("<s>")

{'input_ids': [0, 0, 2], 'attention_mask': [1, 1, 1]}

In [ ]:
' </s> en_XX The US President was to receive Iraq i Prime Minister No uri Al Malik i Friday , November 1 , 2013 in an effort to se ek US assistance in fighting the worst wa ve of violence in five years'